# Philadelphia Housing Analysis Project

## Introduction

Permits are required in most jurisdictions whenever developers or homeowners want to alter or remodel their homes, construct new buildings, and/or demolish existing structures.  The type of permit required depends on both the local jurisdiction and the scope of work performed. In Philadelphia, Pennsylvania, permits are issued by The Department of Licenses & Inspections and the data for those permits are publicly available at https://www.opendataphilly.org/.  For this analysis, two files downloaded from [the Licenses and Inspections Building and Zoning Permits page](https://www.opendataphilly.org/dataset/licenses-and-inspections-building-permits) of this website will be used, listing information for permits from 2007 to the present.  

The other data to be utilized in this analysis will be housing price data made available by Zillow at https://www.zillow.com/research/data/.  The data from Zillow is based on the Zillow Home Value Index (ZHVI) when represents the “typical home value for the region” as defined in Zillow's [methodology guide](https://www.zillow.com/research/zhvi-methodology-2019-highlights-26221/).  

The goal of this analysis is to determine whether the number of permits issued in a given zip code within a given year is correlated to changes of property values within the zip code, years into the future.  The assumption going in is that building permits that better the buildings within a zip code should have some impact on the future housing prices.

The limitations for this analysis include the following:
- The permit data available ranges back from 2007, which only gives 15 years of analysis.  
- The housing market crash of 2008 occured during the short timeframe available within our data.  This crash would be assumed to have affected housing price data and may overshadow any correlations between housing prices and permit issuance.
- Not all renovation work on a building requires a permit, which would limit the data available within our permit dataset.  In addition, some work is occasionally performed without proper permitting, which would again limit data available in our dataset.

With all of that understood, we can begin exploring the data and performing the analysis. 

## Permit Data

### Data Exploration

Exploring the permit data, first let's look at the column names within our dataset.

In [58]:
import csv    

with open('data_files/permits.csv', 'r') as infile:
    reader = csv.DictReader(infile)
    fieldnames = reader.fieldnames
    
fieldnames

['objectid',
 'permitnumber',
 'addressobjectid',
 'parcel_id_num',
 'permittype',
 'permitdescription',
 'commercialorresidential',
 'typeofwork',
 'approvedscopeofwork',
 'permitissuedate',
 'status',
 'applicanttype',
 'contractorname',
 'contractoraddress1',
 'contractoraddress2',
 'contractorcity',
 'contractorstate',
 'contractorzip',
 'mostrecentinsp',
 'opa_account_num',
 'address',
 'unit_type',
 'unit_num',
 'zip',
 'censustract',
 'council_district',
 'opa_owner',
 'systemofrecord',
 'geocode_x',
 'geocode_y',
 'posse_jobid',
 'lat',
 'lng']

In [59]:
import pandas as pd
pd.options.display.max_columns = None

# Note, column dtypes defined due to errors when left undefined
permits = pd.read_csv('data_files/permits.csv', dtype={'permitnumber':str, 'parcel_id_num':str})
permits.head()

,objectid,permitnumber,addressobjectid,parcel_id_num,permittype,permitdescription,commercialorresidential,typeofwork,approvedscopeofwork,permitissuedate,status,applicanttype,contractorname,contractoraddress1,contractoraddress2,contractorcity,contractorstate,contractorzip,mostrecentinsp,opa_account_num,address,unit_type,unit_num,zip,censustract,council_district,opa_owner,systemofrecord,geocode_x,geocode_y,posse_jobid,lat,lng
0,32,100754,8402,336710,L_FFORM,FAST FORM BUILDING PERMIT,NaN,NaN,NaN,2007-09-04 16:41:02,COMPLETED,APPL,NaN,NaN,NaN,NaN,NaN,NaN,2008-01-29 15:20:00,632172000.0,126 ALEXIS LN,NaN,NaN,19115-3402,344.0,10.0,CHEN ZHI QIANG ZHENG FANG,HANSEN,2.720980e+06,290557.097420,NaN,40.099475,-75.059753
1,2,100029,740935,534979,BP_ADMINST,ADMINISTRATIVE PERMIT,NaN,CO,CERTIFICATE OF OCCUPANCY,2007-08-30 16:37:33,COMPLETED,OWNER,J J DELUCA CO INC,"760 W SPROUL ROAD, STE 300",NaN,SPRINGFIELD,PA,19064-4001,2007-09-28 18:20:00,888112914.0,815-37 ARCH ST,NaN,621,19107-2403,2.0,1.0,CHIANG MAGGIE,HANSEN,2.696151e+06,236716.444730,NaN,39.953782,-75.154127
2,1017,240571,360472,132411,ZP_USE,USE PERMIT,NaN,SFRED,CREATE MOTHER IN LAW SUITE IN BASEMENT OF SFD.,2009-10-02 14:42:21,COMPLETED,OWNER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,652264500.0,8759 JACKSON ST,NaN,NaN,19136-2141,349.0,6.0,KAMDAR PRAPTI M KAMDAR MEHUL PANKAJ,HANSEN,2.735362e+06,271794.924559,NaN,40.046782,-75.010448
3,13,100212,17535,111870,PP_PLUMBNG,PLUMBING PERMIT,NaN,EZPLUM,"AREA DRAIN, BASEMENT: LT, WC, LAV, HWH, 1ST FL...",2007-08-31 13:56:52,COMPLETED,CONTRACTOR,ANTHONY S. RACHUBA JR,1660 FOULKROD STREET,NaN,PHILADELPHIA,PA,19124-,2008-05-20 13:32:00,361021600.0,1909 ANNIN ST,NaN,NaN,19146-2907,21.0,2.0,GORDON JESSICA L,HANSEN,2.690311e+06,230662.085428,NaN,39.937639,-75.175576
4,43,100984,419501,188921,L_FFORM,FAST FORM BUILDING PERMIT,NaN,NaN,NaN,2007-09-05 16:24:13,COMPLETED,APPL,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-17 15:45:00,291108000.0,2632 MASTER ST,NaN,NaN,19121-0000,138.0,5.0,MOSHER JENNIFER LYNN LANGHAM MICHAEL CHARLES,HANSEN,2.688834e+06,244955.524768,NaN,39.976977,-75.179360


The relevant columnns for this analysis would be the following:
- Zipcode of Proposed Work ('zip' column)
- Permit Description ('permitdescription' column)
- Permit Issue Date ('permitissuedate' column)

The permit description column:

In [60]:
permits['permitdescription'].value_counts()

PLUMBING PERMIT                 97518
ELECTRICAL PERMIT               70452
ALTERATION PERMIT               66810
MECHANICAL PERMIT               27327
ZONING/USE PERMIT               21541
USE PERMIT                      19030
ZONING PERMIT                   15495
SUPPRESSION PERMIT              14375
ADMINISTRATIVE PERMIT           14279
NEW CONSTRUCTION PERMIT          9461
DEMOLITION PERMIT                7478
OPERATIONS PERMIT                4947
ADDITION PERMIT                  4751
FAST FORM BUILDING PERMIT        4376
SIGN PERMIT                      2895
ZONING ADMIN REVIEW              1382
RESIDENTIAL BUILDING PERMIT       164
COMMERCIAL BUILDING PERMIT         93
GENERAL PERMIT MINOR               48
MECHANICAL / FUEL GAS PERMIT       27
FIRE SUPPRESSION PERMIT            18
SITE / UTILITY PERMIT               8
GENERAL PERMIT                      2
5-YEAR TANK TESTING                 1
Name: permitdescription, dtype: int64

### Data Cleaning

Insert text here.

We'll use these columns because...

We'll leave out these because...

In [61]:
permit_col_list = [
    'ALTERATION PERMIT',
    'ZONING/USE PERMIT',
    'USE PERMIT',
    'ZONING PERMIT',                   
    'NEW CONSTRUCTION PERMIT',          
    'ADDITION PERMIT',                  
    'FAST FORM BUILDING PERMIT',        
    'RESIDENTIAL BUILDING PERMIT',       
    'COMMERCIAL BUILDING PERMIT'
]

In [62]:
# Remove rows with permitdescription not in list above
permits = permits[permits['permitdescription'].isin(permit_col_list)]

Zip code

In [63]:
# Create zipcode column with only 5 digits, removing the trailing digits
permits['zipcode'] = permits['zip'].str.split('-', expand=True)[0]

In [64]:
permits['zipcode'].unique()

array(['19115', '19136', '19121', '19102', '19111', '19140', '19120',
       '19145', '19107', '19134', '19133', '19147', '19146', '19122',
       '19129', '19130', '19116', '19125', '19104', '19123', '19139',
       '19144', '19141', '19148', '19143', '19132', '19153', '19131',
       '19103', '19149', '19106', '19154', '19135', '19127', '19118',
       '19114', '19142', '19138', '19119', '19124', '19128', '19137',
       '19126', '19152', '19151', '19150', '19112', '19109', ' ', '19108',
       '19110', '12345', '12123'], dtype=object)

List of zipcodes from: https://www.zillow.com/browse/homes/pa/philadelphia-county/

In [65]:
list_of_zipcodes = [
    '19120','19124','19111','19143','19134','19149','19148','19104','19140','19145',
    '19139','19131','19144','19146','19147','19128','19135','19115','19141','19136',
    '19154','19152','19151','19116','19121','19132','19114','19138','19142','19119',
    '19125','19133','19130','19150','19103','19122','19126','19123','19107','19153',
    '19106','19129','19118','19137','19102','19127','19019','19093','19092','19101',
    '19099','19105','19109','19108','19110','19112','19160','19155','19162','19161',
    '19171','19170','19173','19172','19176','19175','19178','19177','19181','19179',
    '19183','19182','19185','19184','19188','19187','19192','19191','19194','19193',
    '19196','19195','19244','19197','19255','19190']

In [66]:
permits = permits[permits['zipcode'].isin(list_of_zipcodes)]

Extract the year

In [67]:
# Convert 'permitissuedate' to datetime and extract the year in a new column
permits['permitissuedate'] = pd.to_datetime(permits['permitissuedate'])
permits['permitissueyear'] = pd.DatetimeIndex(permits['permitissuedate']).year

### Group Permit Data by Zipcode and Year

More text.

In [68]:
npermits1_cleaned = permits.groupby(['zipcode','permitissueyear'])['zipcode'].count().reset_index(name='no_permits')
npermits1_cleaned.head()

,zipcode,permitissueyear,no_permits
0,19102,2007,175
1,19102,2008,155
2,19102,2009,117
3,19102,2010,113
4,19102,2011,132


### Convert Data into New DataFrame

In [70]:
permit1_data = []

for zipcode in npermits1_cleaned['zipcode'].unique():
    zip_dict = {}
    zip_dict['zipcode'] = zipcode
    for index,row in npermits1_cleaned[npermits1_cleaned['zipcode']==zipcode].iterrows():
        zip_dict["np_"+str(row['permitissueyear'])] = row['no_permits']
    permit1_data.append(zip_dict)

In [71]:
pdf1 = pd.DataFrame(permit1_data)
pdf1.head()

,zipcode,np_2007,np_2008,np_2009,np_2010,np_2011,np_2012,np_2013,np_2014,np_2015
0,19102,175,155,117.0,113,132.0,128.0,146.0,143,221.0
1,19103,768,696,486.0,493,583.0,629.0,652.0,733,733.0
2,19104,878,742,638.0,559,609.0,764.0,662.0,703,790.0
3,19106,427,374,224.0,248,322.0,324.0,350.0,368,348.0
4,19107,486,461,341.0,351,351.0,399.0,407.0,375,384.0


### Repeat Above Process for Second File of Permit Data

In [20]:
permits2 = pd.read_csv('data_files/permits-2.csv')
permits2.head()

,objectid,permitnumber,addressobjectid,parcel_id_num,permittype,permitdescription,commercialorresidential,typeofwork,approvedscopeofwork,permitissuedate,...,zip,censustract,council_district,opa_owner,systemofrecord,geocode_x,geocode_y,posse_jobid,lat,lng
0,28,1007062,15597742.0,481868,FIRE SUPPRESSION,FIRE SUPPRESSION PERMIT,RESIDENTIAL,NEW CONSTRUCTION,FOR THE INSTALLATION OF A FIRE SUPPRESSION SYS...,2019-10-03 11:01:00,...,19122-4325,141.0,5.0,STEIN MARC,ECLIPSE,2.697602e+06,243701.739648,191894414.0,39.972832,-75.148217
1,29,1007180,603633.0,333973,BP_FIRESUP,SUPPRESSION PERMIT,NaN,RELOCA,RELOCATE 145 EXISTING SPRINKLER HEADS,2019-09-11 11:13:33,...,19144-3313,241.0,8.0,GERMANTOWN EQUITIES LLC A NEW JERSEY LTD LIAB CO,HANSEN,2.688261e+06,265365.103270,NaN,40.033026,-75.179282
2,30,1007380,156586952.0,199356,PLUMBING,PLUMBING PERMIT,COMMERCIAL,NEW CONSTRUCTION OR ADDITIONS,INSTALL 18 FIXTURES AS PER 2004 PPC,2019-09-04 10:46:13,...,19147-1818,15.0,2.0,606 LOFTS LLC,ECLIPSE,2.693602e+06,232748.621978,226806589.0,39.943100,-75.163628
3,31,1007520,499505.0,105368,BP_ALTER,ALTERATION PERMIT,NaN,MINOR,"ROOF REPLACEMENT, SINGLE PLY 1020 SF\r\nVINYL ...",2019-09-04 14:35:40,...,19140-2531,197.0,7.0,ROSADO MIGUEL A,HANSEN,2.700610e+06,259657.774849,NaN,40.016371,-75.135806
4,33,1007549,158279237.0,432779,BUILDING,COMMERCIAL BUILDING PERMIT,COMMERCIAL,ADDITION AND/OR ALTERATION,FOR LEVEL II ALTERATIONS TO EXISTING INTERIOR ...,2019-10-09 09:53:00,...,19107-1138,376.0,1.0,NaN,ECLIPSE,2.694791e+06,238306.023900,217611319.0,39.958254,-75.158807


In [46]:
permits2['permitdescription'].value_counts()

ALTERATION PERMIT              33476
ZONING PERMIT                  21795
RESIDENTIAL BUILDING PERMIT    17902
ZONING/USE PERMIT              12921
COMMERCIAL BUILDING PERMIT      9710
USE PERMIT                      9651
NEW CONSTRUCTION PERMIT         4984
ADDITION PERMIT                 1781
Name: permitdescription, dtype: int64

Values here are the same.  Do the same manips:

In [72]:
# Remove rows with permitdescription not in list above
permits2 = permits2[permits2['permitdescription'].isin(permit_col_list)]

# Create zipcode column with only 5 digits, removing the trailing digits and remove rows with erroneous zipcodes
permits2['zipcode'] = permits2['zip'].str.split('-', expand=True)[0]
permits2 = permits2[permits2['zipcode'].isin(list_of_zipcodes)]

# Convert 'permitissuedate' to datetime and extract the year in a new column
permits2['permitissuedate'] = pd.to_datetime(permits2['permitissuedate'])
permits2['permitissueyear'] = pd.DatetimeIndex(permits2['permitissuedate']).year

In [74]:
npermits2_cleaned = permits2.groupby(['zipcode','permitissueyear'])['zipcode'].count().reset_index(name='no_permits')
npermits2_cleaned

,zipcode,permitissueyear,no_permits
0,19102,2016,202
1,19102,2017,181
2,19102,2018,173
3,19102,2019,146
4,19102,2020,70
...,...,...,...
347,19154,2022,36
348,19192,2016,2
349,19192,2019,3
350,19192,2020,2


In [76]:
permit2_data = []

for zipcode in npermits2_cleaned['zipcode'].unique():
    zip_dict = {}
    zip_dict['zipcode'] = zipcode
    for index,row in npermits2_cleaned[npermits2_cleaned['zipcode']==zipcode].iterrows():
        zip_dict["np_"+str(row['permitissueyear'])] = row['no_permits']
    permit2_data.append(zip_dict)

In [77]:
pdf2 = pd.DataFrame(permit2_data)
pdf2.head()

,zipcode,np_2016,np_2017,np_2018,np_2019,np_2020,np_2021,np_2022
0,19102,202.0,181.0,173.0,146.0,70.0,88,50.0
1,19103,680.0,665.0,672.0,593.0,399.0,402,239.0
2,19104,743.0,738.0,940.0,947.0,698.0,849,388.0
3,19105,NaN,NaN,NaN,NaN,2.0,4,NaN
4,19106,368.0,384.0,417.0,325.0,177.0,234,134.0


## Zillow

some text.

In [25]:
zillow = pd.read_csv('data_files/SFD.csv')
zillow.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,...,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,...,1996560.0,1998803.0,1996185.0,2011349.0,2040228.0,2082630.0,2111986.0,2124199.0,2137132.0,2134425.0
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,372531.0,...,695951.0,701019.0,705067.0,708269.0,709245.0,713447.0,716484.0,723778.0,729846.0,734328.0
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,...,2946803.0,2944829.0,2936001.0,2943103.0,2946178.0,2977571.0,3004616.0,3054765.0,3081294.0,3107469.0
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,224507.0,...,428636.0,435717.0,441454.0,447545.0,457074.0,468753.0,482521.0,493971.0,501506.0,503792.0
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,491164.0,...,936047.0,943466.0,949045.0,954533.0,955605.0,960998.0,965042.0,974178.0,982504.0,988065.0


In [91]:
zillow_philly = zillow[(zillow['City'] == 'Philadelphia') & (zillow['State'] == 'PA')]
# zillow_philly = zillow_philly.rename(columns={"RegionName": "zipcode"})
zillow_philly

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,2000-06-30,2000-07-31,2000-08-31,2000-09-30,2000-10-31,2000-11-30,2000-12-31,2001-01-31,2001-02-28,2001-03-31,2001-04-30,2001-05-31,2001-06-30,2001-07-31,2001-08-31,2001-09-30,2001-10-31,2001-11-30,2001-12-31,2002-01-31,2002-02-28,2002-03-31,2002-04-30,2002-05-31,2002-06-30,2002-07-31,2002-08-31,2002-09-30,2002-10-31,2002-11-30,2002-12-31,2003-01-31,2003-02-28,2003-03-31,2003-04-30,2003-05-31,2003-06-30,2003-07-31,2003-08-31,2003-09-30,2003-10-31,2003-11-30,2003-12-31,2004-01-31,2004-02-29,2004-03-31,2004-04-30,2004-05-31,2004-06-30,2004-07-31,2004-08-31,2004-09-30,2004-10-31,2004-11-30,2004-12-31,2005-01-31,2005-02-28,2005-03-31,2005-04-30,2005-05-31,2005-06-30,2005-07-31,2005-08-31,2005-09-30,2005-10-31,2005-11-30,2005-12-31,2006-01-31,2006-02-28,2006-03-31,2006-04-30,2006-05-31,2006-06-30,2006-07-31,2006-08-31,2006-09-30,2006-10-31,2006-11-30,2006-12-31,2007-01-31,2007-02-28,2007-03-31,2007-04-30,2007-05-31,2007-06-30,2007-07-31,2007-08-31,2007-09-30,2007-10-31,2007-11-30,2007-12-31,2008-01-31,2008-02-29,2008-03-31,2008-04-30,2008-05-31,2008-06-30,2008-07-31,2008-08-31,2008-09-30,2008-10-31,2008-11-30,2008-12-31,2009-01-31,2009-02-28,2009-03-31,2009-04-30,2009-05-31,2009-06-30,2009-07-31,2009-08-31,2009-09-30,2009-10-31,2009-11-30,2009-12-31,2010-01-31,2010-02-28,2010-03-31,2010-04-30,2010-05-31,2010-06-30,2010-07-31,2010-08-31,2010-09-30,2010-10-31,2010-11-30,2010-12-31,2011-01-31,2011-02-28,2011-03-31,2011-04-30,2011-05-31,2011-06-30,2011-07-31,2011-08-31,2011-09-30,2011-10-31,2011-11-30,2011-12-31,2012-01-31,2012-02-29,2012-03-31,2012-04-30,2012-05-31,2012-06-30,2012-07-31,2012-08-31,2012-09-30,2012-10-31,2012-11-30,2012-12-31,2013-01-31,2013-02-28,2013-03-31,2013-04-30,2013-05-31,2013-06-30,2013-07-31,2013-08-31,2013-09-30,2013-10-31,2013-11-30,2013-12-31,2014-01-31,2014-02-28,2014-03-31,2014-04-30,2014-05-31,2014-06-30,2014-07-31,2014-08-31,2014-09-30,2014-10-31,2014-11-30,2014-12-31,2015-01-31,2015-02-28,2015-03-31,2015-04-30,2015-05-31,2015-06-30,2015-07-31,2015-08-31,2015-09-30,2015-10-31,2015-11-30,2015-12-31,2016-01-31,2016-02-29,2016-03-31,2016-04-30,2016-05-31,2016-06-30,2016-07-31,2016-08-31,2016-09-30,2016-10-31,2016-11-30,2016-12-31,2017-01-31,2017-02-28,2017-03-31,2017-04-30,2017-05-31,2017-06-30,2017-07-31,2017-08-31,2017-09-30,2017-10-31,2017-11-30,2017-12-31,2018-01-31,2018-02-28,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31
124,65810,126,19143,Zip,PA,PA,Philadelphia,Philadelphia-Camden-Wilmington,Philadelphia County,60105.0,60432.0,60552.0,60938.0,61540.0,62272.0,62851.0,63378.0,63732.0,63997.0,64146.0,64595.0,65103.0,65287.0,65230.0,65451.0,65612.0,66117.0,66419.0,66951.0,66983.0,67237.0,67482.0,67976.0,67734.0,67462.0,67071.0,67164.0,67387.0,67771.0,67878.0,67790.0,67662.0,67626.0,67763.0,67924.0,68550.0,69512.0,70409.0,71170.0,70594.0,69880.0,69010.0,68210.0,67931.0,67920.0,68416.0,68467.0,68132.0,67764.0,67402.0,66834.0,67492.0,68658.0,71354.0,73563.0,75929.0,76951.0,77461.0,77719.0,78698.0,80616.0,83012.0,85368.0,87460.0,89451.0,91207.0,92921.0,94104.0,95098.0,95771.0,96272.0,96446.0,96364.0,96576.0,97168.0,98007.0,99131.0,100418.0,101573.0,102056.0,102199.0,102150.0,102583.0,103579.0,104578.0,105566.0,106153.0,106973.0,107698.0,108333.0,108692.0,108747.0,109107.0,109770.0,110677.0,110772.0,111107.0,110732.0,110891.0,110396.0,110238.0,109418

In [86]:
zillow_philly.shape

(46, 280)

Group by explanation

In [94]:
groupby_dict = {}

year_list = [str(x) for x in range(2007,2022)]

for col in zillow_philly.columns:
    for year in year_list:
        if year in col:
            groupby_dict[col] = 'hv_'+year
groupby_dict['RegionName'] = 'RegionName'

zillow_philly_condensed = zillow_philly.groupby(groupby_dict, axis = 1).mean()

In [95]:
zillow_philly_condensed.head()

,RegionName,hv_2007,hv_2008,hv_2009,hv_2010,hv_2011,hv_2012,hv_2013,hv_2014,hv_2015,hv_2016,hv_2017,hv_2018,hv_2019,hv_2020,hv_2021
124,19143.0,107489.416667,108810.833333,101106.250000,99132.750000,90704.000000,78566.333333,76493.583333,76854.416667,77649.583333,77601.000000,82145.416667,103273.000000,122495.750000,145343.750000,171277.166667
246,19111.0,184306.250000,178325.000000,167704.666667,167685.333333,159020.500000,155220.833333,153293.000000,152308.666667,155516.416667,158646.666667,170367.000000,183703.416667,200616.083333,214283.666667,248063.416667
337,19124.0,98994.333333,97981.666667,93005.666667,92144.833333,86874.916667,83903.916667,79047.500000,80259.416667,83230.083333,85523.916667,92259.500000,103562.833333,115682.750000,129448.750000,150209.166667
422,19120.0,110987.333333,109877.750000,104370.333333,104722.750000,99627.333333,93638.333333,86917.166667,87473.083333,89573.083333,91115.750000,98473.750000,108718.416667,120119.750000,133145.666667,155033.333333
508,19104.0,126901.833333,131180.416667,135186.416667,137686.000000,116803.416667,90868.916667,102550.583333,105260.250000,123812.250000,125861.250000,134896.500000,160664.250000,173954.916667,190482.166667,216264.666667


In [30]:
zillow_philly_condensed['RegionName'] = zillow_philly_condensed['RegionName'].astype('int').astype('str')

In [31]:
zillow_philly_condensed.head()

,RegionName,hv_2007,hv_2008,hv_2009,hv_2010,hv_2011,hv_2012,hv_2013,hv_2014,hv_2015,hv_2016,hv_2017,hv_2018,hv_2019,hv_2020,hv_2021
124,19143,107489.416667,108810.833333,101106.250000,99132.750000,90704.000000,78566.333333,76493.583333,76854.416667,77649.583333,77601.000000,82145.416667,103273.000000,122495.750000,145343.750000,171277.166667
246,19111,184306.250000,178325.000000,167704.666667,167685.333333,159020.500000,155220.833333,153293.000000,152308.666667,155516.416667,158646.666667,170367.000000,183703.416667,200616.083333,214283.666667,248063.416667
337,19124,98994.333333,97981.666667,93005.666667,92144.833333,86874.916667,83903.916667,79047.500000,80259.416667,83230.083333,85523.916667,92259.500000,103562.833333,115682.750000,129448.750000,150209.166667
422,19120,110987.333333,109877.750000,104370.333333,104722.750000,99627.333333,93638.333333,86917.166667,87473.083333,89573.083333,91115.750000,98473.750000,108718.416667,120119.750000,133145.666667,155033.333333
508,19104,126901.833333,131180.416667,135186.416667,137686.000000,116803.416667,90868.916667,102550.583333,105260.250000,123812.250000,125861.250000,134896.500000,160664.250000,173954.916667,190482.166667,216264.666667


In [32]:
zillow_philly_cleaned = zillow_philly_condensed[zillow_philly_condensed['RegionName'].isin(list_of_zipcodes)]
zillow_philly_cleaned = zillow_philly_cleaned.rename(columns={"RegionName": "zipcode"})
zillow_philly_cleaned.head()

,zipcode,hv_2007,hv_2008,hv_2009,hv_2010,hv_2011,hv_2012,hv_2013,hv_2014,hv_2015,hv_2016,hv_2017,hv_2018,hv_2019,hv_2020,hv_2021
124,19143,107489.416667,108810.833333,101106.250000,99132.750000,90704.000000,78566.333333,76493.583333,76854.416667,77649.583333,77601.000000,82145.416667,103273.000000,122495.750000,145343.750000,171277.166667
246,19111,184306.250000,178325.000000,167704.666667,167685.333333,159020.500000,155220.833333,153293.000000,152308.666667,155516.416667,158646.666667,170367.000000,183703.416667,200616.083333,214283.666667,248063.416667
337,19124,98994.333333,97981.666667,93005.666667,92144.833333,86874.916667,83903.916667,79047.500000,80259.416667,83230.083333,85523.916667,92259.500000,103562.833333,115682.750000,129448.750000,150209.166667
422,19120,110987.333333,109877.750000,104370.333333,104722.750000,99627.333333,93638.333333,86917.166667,87473.083333,89573.083333,91115.750000,98473.750000,108718.416667,120119.750000,133145.666667,155033.333333
508,19104,126901.833333,131180.416667,135186.416667,137686.000000,116803.416667,90868.916667,102550.583333,105260.250000,123812.250000,125861.250000,134896.500000,160664.250000,173954.916667,190482.166667,216264.666667


## Check out our data

In [33]:
pdf1.head()

,zipcode,np_2007,np_2008,np_2009,np_2010,np_2011,np_2012,np_2013,np_2014,np_2015
0,19102,175,155,117.0,113,132.0,128.0,146.0,143,221.0
1,19103,768,696,486.0,493,583.0,629.0,652.0,733,733.0
2,19104,878,742,638.0,559,609.0,764.0,662.0,703,790.0
3,19106,427,374,224.0,248,322.0,324.0,350.0,368,348.0
4,19107,486,461,341.0,351,351.0,399.0,407.0,375,384.0


In [34]:
pdf2.head()

,zipcode,np_2016,np_2017,np_2018,np_2019,np_2020,np_2021,np_2022
0,19102,202.0,181.0,173.0,146.0,70.0,88,50.0
1,19103,680.0,665.0,672.0,593.0,399.0,402,239.0
2,19104,743.0,738.0,940.0,947.0,698.0,849,388.0
3,19105,NaN,NaN,NaN,NaN,2.0,4,NaN
4,19106,368.0,384.0,417.0,325.0,177.0,234,134.0


In [35]:
zillow_philly_cleaned.head()

,zipcode,hv_2007,hv_2008,hv_2009,hv_2010,hv_2011,hv_2012,hv_2013,hv_2014,hv_2015,hv_2016,hv_2017,hv_2018,hv_2019,hv_2020,hv_2021
124,19143,107489.416667,108810.833333,101106.250000,99132.750000,90704.000000,78566.333333,76493.583333,76854.416667,77649.583333,77601.000000,82145.416667,103273.000000,122495.750000,145343.750000,171277.166667
246,19111,184306.250000,178325.000000,167704.666667,167685.333333,159020.500000,155220.833333,153293.000000,152308.666667,155516.416667,158646.666667,170367.000000,183703.416667,200616.083333,214283.666667,248063.416667
337,19124,98994.333333,97981.666667,93005.666667,92144.833333,86874.916667,83903.916667,79047.500000,80259.416667,83230.083333,85523.916667,92259.500000,103562.833333,115682.750000,129448.750000,150209.166667
422,19120,110987.333333,109877.750000,104370.333333,104722.750000,99627.333333,93638.333333,86917.166667,87473.083333,89573.083333,91115.750000,98473.750000,108718.416667,120119.750000,133145.666667,155033.333333
508,19104,126901.833333,131180.416667,135186.416667,137686.000000,116803.416667,90868.916667,102550.583333,105260.250000,123812.250000,125861.250000,134896.500000,160664.250000,173954.916667,190482.166667,216264.666667


In [36]:
zpchc = zillow_philly_cleaned.copy()
zpchc.head()

,zipcode,hv_2007,hv_2008,hv_2009,hv_2010,hv_2011,hv_2012,hv_2013,hv_2014,hv_2015,hv_2016,hv_2017,hv_2018,hv_2019,hv_2020,hv_2021
124,19143,107489.416667,108810.833333,101106.250000,99132.750000,90704.000000,78566.333333,76493.583333,76854.416667,77649.583333,77601.000000,82145.416667,103273.000000,122495.750000,145343.750000,171277.166667
246,19111,184306.250000,178325.000000,167704.666667,167685.333333,159020.500000,155220.833333,153293.000000,152308.666667,155516.416667,158646.666667,170367.000000,183703.416667,200616.083333,214283.666667,248063.416667
337,19124,98994.333333,97981.666667,93005.666667,92144.833333,86874.916667,83903.916667,79047.500000,80259.416667,83230.083333,85523.916667,92259.500000,103562.833333,115682.750000,129448.750000,150209.166667
422,19120,110987.333333,109877.750000,104370.333333,104722.750000,99627.333333,93638.333333,86917.166667,87473.083333,89573.083333,91115.750000,98473.750000,108718.416667,120119.750000,133145.666667,155033.333333
508,19104,126901.833333,131180.416667,135186.416667,137686.000000,116803.416667,90868.916667,102550.583333,105260.250000,123812.250000,125861.250000,134896.500000,160664.250000,173954.916667,190482.166667,216264.666667


In [37]:
for i in range(0,len(year_list)):
    year1 = year_list[i]
    j = i+1
    while j < len(year_list):
        year2 = year_list[j]
        zpchc['hg_'+year1+'-'+year2] = 100 * (zpchc['hv_'+year2] - zpchc['hv_'+year1]) / zpchc['hv_'+year1]
        j += 1
    
len(zpchc.columns)

/var/folders/1b/w1kczxkd6c35q23928bnt7ch0000gn/T/ipykernel_18206/3865440580.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  zpchc['hg_'+year1+'-'+year2] = 100 * (zpchc['hv_'+year2] - zpchc['hv_'+year1]) / zpchc['hv_'+year1]
/var/folders/1b/w1kczxkd6c35q23928bnt7ch0000gn/T/ipykernel_18206/3865440580.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  zpchc['hg_'+year1+'-'+year2] = 100 * (zpchc['hv_'+year2] - zpchc['hv_'+year1]) / zpchc['hv_'+year1]
/var/folders/1b/w1kczxkd6c35q23928bnt7ch0000gn/T/ipykernel_18206/3865440580.py

121

In [38]:
zpchc.head()

,zipcode,hv_2007,hv_2008,hv_2009,hv_2010,hv_2011,hv_2012,hv_2013,hv_2014,hv_2015,hv_2016,hv_2017,hv_2018,hv_2019,hv_2020,hv_2021,hg_2007-2008,hg_2007-2009,hg_2007-2010,hg_2007-2011,hg_2007-2012,hg_2007-2013,hg_2007-2014,hg_2007-2015,hg_2007-2016,hg_2007-2017,hg_2007-2018,hg_2007-2019,hg_2007-2020,hg_2007-2021,hg_2008-2009,hg_2008-2010,hg_2008-2011,hg_2008-2012,hg_2008-2013,hg_2008-2014,hg_2008-2015,hg_2008-2016,hg_2008-2017,hg_2008-2018,hg_2008-2019,hg_2008-2020,hg_2008-2021,hg_2009-2010,hg_2009-2011,hg_2009-2012,hg_2009-2013,hg_2009-2014,hg_2009-2015,hg_2009-2016,hg_2009-2017,hg_2009-2018,hg_2009-2019,hg_2009-2020,hg_2009-2021,hg_2010-2011,hg_2010-2012,hg_2010-2013,hg_2010-2014,hg_2010-2015,hg_2010-2016,hg_2010-2017,hg_2010-2018,hg_2010-2019,hg_2010-2020,hg_2010-2021,hg_2011-2012,hg_2011-2013,hg_2011-2014,hg_2011-2015,hg_2011-2016,hg_2011-2017,hg_2011-2018,hg_2011-2019,hg_2011-2020,hg_2011-2021,hg_2012-2013,hg_2012-2014,hg_2012-2015,hg_2012-2016,hg_2012-2017,hg_2012-2018,hg_2012-2019,hg_2012-2020,hg_2012-2021,hg_2013-2014,hg_2013-2015,hg_2013-2016,hg_2013-2017,hg_2013-2018,hg_2013-2019,hg_2013-2020,hg_2013-2021,hg_2014-2015,hg_2014-2016,hg_2014-2017,hg_2014-2018,hg_2014-2019,hg_2014-2020,hg_2014-2021,hg_2015-2016,hg_2015-2017,hg_2015-2018,hg_2015-2019,hg_2015-2020,hg_2015-2021,hg_2016-2017,hg_2016-2018,hg_2016-2019,hg_2016-2020,hg_2016-2021,hg_2017-2018,hg_2017-2019,hg_2017-2020,hg_2017-2021,hg_2018-2019,hg_2018-2020,hg_2018-2021,hg_2019-2020,hg_2019-2021,hg_2020-2021
124,19143,107489.416667,108810.833333,101106.250000,99132.750000,90704.000000,78566.333333,76493.583333,76854.416667,77649.583333,77601.000000,82145.416667,103273.000000,122495.750000,145343.750000,171277.166667,1.229346,-5.938414,-7.774409,-15.615878,-26.907843,-28.836172,-28.500480,-27.760718,-27.805916,-23.578135,-3.922634,13.960754,35.216800,59.343284,-7.080713,-8.894412,-16.640653,-27.795486,-29.700397,-29.368782,-28.638003,-28.682653,-24.506215,-5.089414,12.576796,33.574705,57.408193,-1.951907,-10.288434,-22.293297,-24.343368,-23.986483,-23.200016,-23.248068,-18.753374,2.143043,21.155468,43.753477,69.403144,-8.502488,-20.746339,-22.837222,-22.473232,-21.671109,-21.720118,-17.135945,4.176470,23.567388,46.615271,72.775563,-13.381622,-15.666803,-15.268989,-14.392327,-14.445890,-9.435729,13.857162,35.049998,60.239626,88.830886,-2.638217,-2.178944,-1.166848,-1.228686,4.555492,31.446888,55.913793,84.994951,118.003258,0.471717,1.511238,1.447725,7.388637,35.008710,60.138596,90.007768,123.910502,1.034640,0.971425,6.884445,34.374841,59.386741,89.115677,122.859237,-0.062567,5.789900,32.998782,57.754549,87.179047,120.577058,5.856132,33.082048,57.853314,87.296233,120.715154,25.719735,49.120614,76.934704,108.504836,18.613529,40.737414,65.848931,18.652076,39.822946,17.842815
246,19111,184306.250000,178325.000000,167704.666667,167685.333333,159020.500000,155220.833333,153293.000000,152308.666667,155516.416667,158646.666667,170367.000000,183703.416667,200616.083333,214283.666667,248063.416667,-3.245278,-9.007607,-9.018097,-13.719421,-15.781026,-16.827020,-17.361095,-15.620650,-13.922253,-7.563091,-0.327082,8.849311,16.265003,34.593057,-5.955605,-5.966447,-10.825459,-12.956213,-14.037291,-14.589280,-12.790457,-11.035095,-4.462638,3.016076,12.500257,20.164681,39.107482,-0.011528,-5.178250,-7.443939,-8.593480,-9.180424,-7.267687,-5.401162,1.587513,9.539836,19.624628,27.774421,47.916824,-5.167317,-7.433268,-8.582941,-9.169953,-7.256995,-5.390255,1.599226,9.552465,19.638420,27.789153,47.933878,-2.389419,-3.601737,-4.220735,-2.203542,-0.235085,7.135244,15.521846,26.157372,34.752228,55.994615,-1.241994,-1.876144,0.190428,2.207071,9.757818,18.349717,29.245591,38.050842,59.813223,-0.642125,1.450436,3.492440,11.138147,19.838099,30.871001,39.786987,61.823056,2.106085,4.161287,11.856406,20.612583,31.716788,40.690396,62.868878,2.012810,9.549206,18.124775,28.999939,37.788454,59.509473,7.387696,15.794060,26.454648,35.069757,56.362199,7.828052,17.755248,25.777684,45.605321,9.206506,16.646

In [39]:
pdf = pd.merge(pdf1, pdf2, on='zipcode')
housing_data = pd.merge(pdf, zpchc, on='zipcode')
housing_data.set_index('zipcode', inplace=True)
housing_data.head()

,np_2007,np_2008,np_2009,np_2010,np_2011,np_2012,np_2013,np_2014,np_2015,np_2016,np_2017,np_2018,np_2019,np_2020,np_2021,np_2022,hv_2007,hv_2008,hv_2009,hv_2010,hv_2011,hv_2012,hv_2013,hv_2014,hv_2015,hv_2016,hv_2017,hv_2018,hv_2019,hv_2020,hv_2021,hg_2007-2008,hg_2007-2009,hg_2007-2010,hg_2007-2011,hg_2007-2012,hg_2007-2013,hg_2007-2014,hg_2007-2015,hg_2007-2016,hg_2007-2017,hg_2007-2018,hg_2007-2019,hg_2007-2020,hg_2007-2021,hg_2008-2009,hg_2008-2010,hg_2008-2011,hg_2008-2012,hg_2008-2013,hg_2008-2014,hg_2008-2015,hg_2008-2016,hg_2008-2017,hg_2008-2018,hg_2008-2019,hg_2008-2020,hg_2008-2021,hg_2009-2010,hg_2009-2011,hg_2009-2012,hg_2009-2013,hg_2009-2014,hg_2009-2015,hg_2009-2016,hg_2009-2017,hg_2009-2018,hg_2009-2019,hg_2009-2020,hg_2009-2021,hg_2010-2011,hg_2010-2012,hg_2010-2013,hg_2010-2014,hg_2010-2015,hg_2010-2016,hg_2010-2017,hg_2010-2018,hg_2010-2019,hg_2010-2020,hg_2010-2021,hg_2011-2012,hg_2011-2013,hg_2011-2014,hg_2011-2015,hg_2011-2016,hg_2011-2017,hg_2011-2018,hg_2011-2019,hg_2011-2020,hg_2011-2021,hg_2012-2013,hg_2012-2014,hg_2012-2015,hg_2012-2016,hg_2012-2017,hg_2012-2018,hg_2012-2019,hg_2012-2020,hg_2012-2021,hg_2013-2014,hg_2013-2015,hg_2013-2016,hg_2013-2017,hg_2013-2018,hg_2013-2019,hg_2013-2020,hg_2013-2021,hg_2014-2015,hg_2014-2016,hg_2014-2017,hg_2014-2018,hg_2014-2019,hg_2014-2020,hg_2014-2021,hg_2015-2016,hg_2015-2017,hg_2015-2018,hg_2015-2019,hg_2015-2020,hg_2015-2021,hg_2016-2017,hg_2016-2018,hg_2016-2019,hg_2016-2020,hg_2016-2021,hg_2017-2018,hg_2017-2019,hg_2017-2020,hg_2017-2021,hg_2018-2019,hg_2018-2020,hg_2018-2021,hg_2019-2020,hg_2019-2021,hg_2020-2021
zipcode,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
19102,175,155,117.0,113,132.0,128.0,146.0,143,221.0,202.0,181.0,173.0,146.0,70.0,88,50.0,448299.416667,459371.500000,447551.166667,429518.250000,396388.666667,394604.000000,408841.416667,430853.333333,442137.583333,448011.750000,473422.000000,485155.166667,491883.500000,491769.083333,492031.666667,2.469797,-0.166909,-4.189425,-11.579482,-11.977579,-8.801707,-3.891614,-1.374491,-0.064168,5.603974,8.221235,9.722092,9.696570,9.755143,-2.573153,-6.498716,-13.710653,-14.099155,-10.999830,-6.208084,-3.751629,-2.472890,3.058636,5.612814,7.077496,7.052589,7.109750,-4.029241,-11.431654,-11.830416,-8.649235,-3.730933,-1.209601,0.102912,5.780531,8.402168,9.905534,9.879969,9.938640,-7.713196,-8.128700,-4.813959,0.310833,2.938020,4.305638,10.221626,12.953330,14.519814,14.493175,14.554310,-0.450232,3.141550,8.694665,11.541429,13.023350,19.433788,22.393804,24.091212,24.062347,24.128591,3.608026,9.186256,12.045895,13.534518,19.973949,22.947351,24.652436,24.623441,24.689985,5.383974,8.144030,9.580813,15.795998,18.665856,20.311563,20.283578,20.347804,2.619047,3.982426,9.880083,12.603322,14.164952,14.138396,14.199341,1.328583,7.075720,9.729456,11.251230,11.225352,11.284741,5.671782,8.290724,9.792545,9.767006,9.825617,2.478374,3.899586,3.875418,3.930883,1.386842,1.363258,1.417382,-0.023261,0.030122,0.053396
19103,768,696,486.0,493,583.0,629.0,652.0,733,733.0,680.0,665.0,672.0,593.0,399.0,402,239.0,591061.666667,586514.583333,562259.583333,546907.083333,503061.000000,506059.833333,528423.500000,565461.250000,596443.416667,599575.250000,645829.250000,698018.333333,718903.416667,740674.000000,771245.500000,-0.769308,-4.872940,-7.470385,-14.888576,-14.381212,-10.597569,-4.331260,0.910523,1.440388,9.265968,18.095687,21.629173,25.312474,30.484777,-4.135447,-6.753029,-14.228731,-13.717434,-9.904457,-3.589567,1.692854,2.226827,10.113076,19.011249,22.572130,26.283987,31.496389,-2.730500,-10.528693,-9.995339,-6.017876,0.569429,6.079724,6.636733,14.863182,24.145209,27.859700,31.731681,37.168938,-8.017099,-7.468773,-3.379657,3.392563,9.057541,9.630186,18.087564,27.630150,31.448913,35.429586,41.019475,0.596117,5.041635,12.404112,18.562842,19.185397,28.379908,38.754213,42.905814,47.233437,53.310533,4.419174,11.738022,17.860256,18.479123

In [40]:
housing_data.columns

Index(['np_2007', 'np_2008', 'np_2009', 'np_2010', 'np_2011', 'np_2012',
       'np_2013', 'np_2014', 'np_2015', 'np_2016',
       ...
       'hg_2017-2018', 'hg_2017-2019', 'hg_2017-2020', 'hg_2017-2021',
       'hg_2018-2019', 'hg_2018-2020', 'hg_2018-2021', 'hg_2019-2020',
       'hg_2019-2021', 'hg_2020-2021'],
      dtype='object', length=136)

In [41]:
housing_data.corr().loc['np_'+'2015', housing_data.columns.str.match('hc_'+'2015')]

Series([], Name: np_2015, dtype: float64)

In [42]:
for year in year_list:
    print(year, housing_data.corr().loc['np_'+year, housing_data.columns.str.match('hc_'+year)])

2007 Series([], Name: np_2007, dtype: float64)
2008 Series([], Name: np_2008, dtype: float64)
2009 Series([], Name: np_2009, dtype: float64)
2010 Series([], Name: np_2010, dtype: float64)
2011 Series([], Name: np_2011, dtype: float64)
2012 Series([], Name: np_2012, dtype: float64)
2013 Series([], Name: np_2013, dtype: float64)
2014 Series([], Name: np_2014, dtype: float64)
2015 Series([], Name: np_2015, dtype: float64)
2016 Series([], Name: np_2016, dtype: float64)
2017 Series([], Name: np_2017, dtype: float64)
2018 Series([], Name: np_2018, dtype: float64)
2019 Series([], Name: np_2019, dtype: float64)
2020 Series([], Name: np_2020, dtype: float64)
2021 Series([], Name: np_2021, dtype: float64)


In [43]:
for year in year_list:
    print(year, housing_data.corr().loc['np_'+year, housing_data.columns.str.match('hg_'+year)].max())

2007 0.26713905569524565
2008 0.3125169935805612
2009 0.34149604209830553
2010 0.34894420847120344
2011 0.5211479717282079
2012 0.7254072841744843
2013 0.6088791005140183
2014 0.4990090779954933
2015 0.2696688512338107
2016 0.26010419434660814
2017 0.2690360780223099
2018 0.024353517179203553
2019 0.05115521649170723
2020 -0.16831989993508034
2021 nan


# SUM OF PERMITS IN A YEAR - DO THIS!!!

In [44]:
np_cols = housing_data.columns.str.match('np_')
hg_cols = housing_data.columns.str.match('hg_')

corr = housing_data.corr().loc[np_cols, hg_cols]
# corr.style.background_gradient(cmap='coolwarm').set_properties(**{'font-size': '0pt'})

corr.style.background_gradient(cmap='coolwarm', axis=None).set_precision(2)

/var/folders/1b/w1kczxkd6c35q23928bnt7ch0000gn/T/ipykernel_18206/2390366642.py:7: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  corr.style.background_gradient(cmap='coolwarm', axis=None).set_precision(2)


,hg_2007-2008,hg_2007-2009,hg_2007-2010,hg_2007-2011,hg_2007-2012,hg_2007-2013,hg_2007-2014,hg_2007-2015,hg_2007-2016,hg_2007-2017,hg_2007-2018,hg_2007-2019,hg_2007-2020,hg_2007-2021,hg_2008-2009,hg_2008-2010,hg_2008-2011,hg_2008-2012,hg_2008-2013,hg_2008-2014,hg_2008-2015,hg_2008-2016,hg_2008-2017,hg_2008-2018,hg_2008-2019,hg_2008-2020,hg_2008-2021,hg_2009-2010,hg_2009-2011,hg_2009-2012,hg_2009-2013,hg_2009-2014,hg_2009-2015,hg_2009-2016,hg_2009-2017,hg_2009-2018,hg_2009-2019,hg_2009-2020,hg_2009-2021,hg_2010-2011,hg_2010-2012,hg_2010-2013,hg_2010-2014,hg_2010-2015,hg_2010-2016,hg_2010-2017,hg_2010-2018,hg_2010-2019,hg_2010-2020,hg_2010-2021,hg_2011-2012,hg_2011-2013,hg_2011-2014,hg_2011-2015,hg_2011-2016,hg_2011-2017,hg_2011-2018,hg_2011-2019,hg_2011-2020,hg_2011-2021,hg_2012-2013,hg_2012-2014,hg_2012-2015,hg_2012-2016,hg_2012-2017,hg_2012-2018,hg_2012-2019,hg_2012-2020,hg_2012-2021,hg_2013-2014,hg_2013-2015,hg_2013-2016,hg_2013-2017,hg_2013-2018,hg_2013-2019,hg_2013-2020,hg_2013-2021,hg_2014-2015,hg_2014-2016,hg_2014-2017,hg_2014-2018,hg_2014-2019,hg_2014-2020,hg_2014-2021,hg_2015-2016,hg_2015-2017,hg_2015-2018,hg_2015-2019,hg_2015-2020,hg_2015-2021,hg_2016-2017,hg_2016-2018,hg_2016-2019,hg_2016-2020,hg_2016-2021,hg_2017-2018,hg_2017-2019,hg_2017-2020,hg_2017-2021,hg_2018-2019,hg_2018-2020,hg_2018-2021,hg_2019-2020,hg_2019-2021,hg_2020-2021
np_2007,-0.25,0.01,-0.08,-0.21,-0.36,-0.09,0.07,0.19,0.18,0.22,0.27,0.20,0.17,0.05,0.23,0.06,-0.13,-0.33,-0.03,0.14,0.26,0.24,0.28,0.33,0.25,0.21,0.09,-0.22,-0.34,-0.42,-0.10,0.09,0.22,0.21,0.25,0.30,0.22,0.18,0.05,-0.30,-0.38,-0.05,0.15,0.28,0.26,0.30,0.35,0.25,0.21,0.08,-0.37,0.08,0.27,0.40,0.36,0.38,0.41,0.30,0.25,0.13,0.51,0.60,0.60,0.56,0.53,0.48,0.37,0.31,0.20,0.54,0.58,0.48,0.45,0.41,0.27,0.21,0.09,0.37,0.28,0.32,0.30,0.17,0.12,0.02,0.07,0.18,0.20,0.06,0.03,-0.06,0.17,0.17,0.04,0.02,-0.06,0.14,-0.01,-0.02,-0.10,-0.18,-0.13,-0.19,-0.06,-0.19,-0.29
np_2008,-0.26,-0.13,-0.21,-0.36,-0.44,-0.15,0.02,0.14,0.13,0.18,0.25,0.22,0.20,0.12,0.05,-0.11,-0.31,-0.43,-0.09,0.09,0.21,0.19,0.24,0.31,0.28,0.26,0.17,-0.28,-0.44,-0.45,-0.12,0.08,0.22,0.19,0.25,0.33,0.29,0.26,0.16,-0.40,-0.41,-0.05,0.16,0.29,0.25,0.31,0.39,0.35,0.31,0.21,-0.35,0.12,0.33,0.44,0.38,0.43,0.49,0.42,0.38,0.27,0.57,0.66,0.65,0.59,0.59,0.56,0.48,0.43,0.34,0.58,0.60,0.48,0.49,0.47,0.37,0.31,0.21,0.37,0.26,0.34,0.35,0.27,0.21,0.12,0.03,0.22,0.27,0.18,0.13,0.05,0.27,0.25,0.16,0.12,0.05,0.18,0.10,0.06,-0.00,-0.03,-0.03,-0.09,-0.04,-0.13,-0.20
np_2009,-0.29,-0.20,-0.28,-0.44,-0.52,-0.24,-0.09,0.04,0.03,0.11,0.22,0.24,0.25,0.22,-0.01,-0.18,-0.40,-0.51,-0.20,-0.02,0.11,0.09,0.18,0.30,0.32,0.33,0.28,-0.33,-0.52,-0.51,-0.21,-0.02,0.12,0.10,0.19,0.32,0.33,0.34,0.28,-0.47,-0.46,-0.13,0.07,0.20,0.17,0.27,0.40,0.41,0.41,0.34,-0.40,0.05,0.26,0.37,0.31,0.41,0.52,0.50,0.49,0.41,0.53,0.63,0.62,0.55,0.60,0.62,0.58,0.55,0.47,0.55,0.57,0.44,0.52,0.55,0.49,0.44,0.36,0.35,0.23,0.38,0.44,0.39,0.35,0.27,-0.01,0.30,0.39,0.32,0.28,0.20,0.40,0.39,0.30,0.26,0.19,0.28,0.22,0.19,0.12,0.10,0.10,0.04,0.06,-0.01,-0.07
np_2010,-0.24,-0.19,-0.25,-0.35,-0.42,-0.17,-0.04,0.08,0.09,0.13,0.19,0.17,0.16,0.10,-0.03,-0.18,-0.31,-0.41,-0.13,0.02,0.15,0.14,0.19,0.25,0.23,0.22,0.14,-0.29,-0.38,-0.40,-0.12,0.04,0.17,0.16,0.21,0.28,0.25,0.23,0.15,-0.30,-0.35,-0.05,0.12,0.24,0.23,0.27,0.35,0.31,0.29,0.20,-0.33,0.07,0.24,0.36,0.33,0.36,0.42,0.36,0.33,0.24,0.48,0.55,0.56,0.52,0.51,0.49,0.42,0.38,0.30,0.46,0.52,0.43,0.43,0.42,0.33,0.29,0.20,0.35,0.27,0.32,0.33,0.25,0.21,0.14,0.06,0.21,0.25,0.17,0.14,0.07,0.21,0.22,0.14,0.12,0.06,0.18,0.10,0.08,0.02,-0.02,-0.00,-0.06,0.01,-0.08,-0.16
np_2011,-0.20,-0.09,-0.17,-0.30,-0.36,-0.07,0.07,0.20,0.19,0.24,0.30,0.27,0.24,0.14,0.05,-0.10,-0.27,-0.36,-0.03,0.13,0.26,0.25,0.29,0.36,0.32,0.29,0.19,-0.27,-0.39,-0.38,-0.05,0.13,0.27,0.25,0.31,0.38,0.33,0.29,0.18,-0.34,-0.33,0.03,0.21,0.35,0.32,0.37,0.44,0.39,0.35,0.22,-0.29,0.18,0.36,0.48,0.44,0.48,0.52,0.45,0.40,0.27,0.58,0.64,0.66,0.61,0.60,0.56,0.48,0.43,0.32,0.52,0.60,0.50,0.51,0.48,0.37,

In [45]:
import matplotlib.pyplot as plt

plt.matshow(housing_data.corr().loc[np_cols, hv_cols])
plt.show()

NameError: name 'hv_cols' is not defined